# **Conversion of 3D image formats**

---

<font size = 4>This notebook facilitates the conversion of original images stored in '.lif' (Leica Image File Format) or '.czi' (Carl Zeiss Image) formats to the widely compatible '.tiff' (Tagged Image File Format).

## **1. Dependencies**
---


### **1.1. Install dependencies**
---
<font size = 4>

In [1]:
#@markdown ##Install dependencies
! pip install -q readlif
! pip install -q SimpleITK
! pip install -q aicsimageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.4/248.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.2/405.2 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.1/195.1 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB

### **1.2. Load key dependencies**
---
<font size = 4>

In [2]:
#@markdown ##Load dependencies

from readlif.reader import LifFile
import numpy as np
import tifffile as tf
import os
import SimpleITK as sitk
from aicsimageio import AICSImage
import shutil
from skimage import io, img_as_uint
from skimage.util import random_noise
import xml.etree.ElementTree as ET
import ipywidgets as widgets
from IPython.display import display

## **2. Initialise the Colab session**
---

### **2.1. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [3]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Follow the instructions.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**<font size = 4> If you cannot see your files, reactivate your session by connecting to your hosted runtime.**


<img width="40%" alt ="Example of image detection with retinanet." src="https://github.com/HenriquesLab/ZeroCostDL4Mic/raw/master/Wiki_files/connect_to_hosted.png"><figcaption> Connect to a hosted runtime. </figcaption>

## **3. Select your paths**

---


<font size = 4> **Paths for original folder and original input image**

<font size = 4>**`base_path`:** this is the folder path where your original image is located in your google drive, and where the output will be saved.

<font size = 4>**`input_file`:** this is the name of your original image file.





 To find the path of the folder containing the respective image, go to your Files on the left of the notebook, navigate to the folder containing your file and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.


In [4]:
#@markdown ##Path of the original images:

# base folder where the GT image is
base_path = "/content/gdrive/MyDrive/Colab Notebooks/res" #@param {type:"string"}

# lif image file name
input_file = "ResonantRestoration.lif" #@param {type:"string"}

output_folder_name = widgets.Text(value='', description='Enter output folder name:')
output_folder_name.style.description_width = '150px'
output_folder_name.layout.width = '600px'

def check_output_folder(base_path, folder_name):
    return folder_name in os.listdir(base_path)

def on_button_clicked(b):
    new_folder_name = output_folder_name.value.strip()
    if new_folder_name:
        if check_output_folder(base_path, new_folder_name):
            print('This folder already exists. Please choose another name.')
            output_folder_name.value = ''

        else:
            print('Folder name is available:', new_folder_name)
    else:
        print('Please provide a name for the output folder.')

button = widgets.Button(description='Check Folder Name')
button.on_click(on_button_clicked)

display(output_folder_name, button)

Text(value='', description='Enter output folder name:', layout=Layout(width='600px'), style=DescriptionStyle(d…

Button(description='Check Folder Name', style=ButtonStyle())

Folder name is available: train_up


## **4. Image format conversion to '.tiff'**
---

<font size = 4>**`lif_to_tiff`,`czi_to_tiff`:** here you select the format of your original image you want to convert to '.tiff' format.

<font size = 4>**`img_dimensions`:** if 'lif_to_tiff' is selected and the '.lif' file contains images with different dimensions, you have the option to selectively extract images based on specific dimensions. Please input the desired dimensions as a pair of x,y numbers (e.g., '2048, 2048'). This setting allows you to extract only those images matching the specified dimensions, but if you wish to extract all images regardless of their dimensions, simply leave this field blank. Disregard the red warning message shown below this field. Please note that providing specific dimensions will filter and extract only the images matching those exact dimensions or none if the dimensions are incorrectly set.

In [7]:
#@markdown ##Images format conversion

def lif_tiff(base_path, input_file, dimensions, output_folder_name):
    '''extract imgs with specified dimensions or extract all imgs from .lif file
     and convert to .tiff imgs'''

    # paths
    lif_path = os.path.join(base_path, input_file)
    lif_file = LifFile(lif_path) # LifFile object using the lif_path

    output_folder = os.path.join(base_path, output_folder_name.value)
    if output_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder)

    # iterating through each image in the .lif file
    for idx, image in enumerate(lif_file.get_iter_image()):
        # check if dimensions are not specified or the img dimensions match the specified dimensions
        if dimensions is None or (image.dims.x, image.dims.y) == dimensions:
            num_slices = image.info['dims'].z # num of slices in the img
            num_channels = image.info['channels'] #num of channels in the img

            # create an empty array to stack imgs (considering variable channels)
            image_stack = np.empty((num_channels * num_slices, image.dims.y, image.dims.x), dtype=np.uint16)

            # loop through slices and channels to populate the img stack
            for z in range(num_slices):
                for c in range(num_channels):
                    frame = np.array(image.get_frame(z=z, c=c)) # get the frame at slice z and channel c
                    image_stack[z * num_channels + c] = frame # assign the frame to the appropriate position in the stack

            voxel_size = image.info['scale'][2::-1] # voxel size info
            resolution = (voxel_size[1], voxel_size[2]) # define resolution based on voxel size

            # output file path for the .tiff img
            output_file_path = os.path.join(output_folder, f'image_{idx + 1}.tiff')

            # save the img stack as a .tiff file with specified metadata
            tf.imwrite(output_file_path, image_stack, resolution=resolution,
                       metadata={'axes': 'ZYX', 'Z': num_channels*num_slices})

    return output_folder # get the path of the output_folder to be used in the subsequent step 2 as input folder


def czi_tiff(base_path, input_file, output_folder_name):
    '''convert .czi img to .tiff img'''

    # paths
    czi_path = os.path.join(base_path, input_file)

    output_folder = os.path.join(base_path, output_folder_name.value)
    if output_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder)

    # read the .czi imge using AICSImage
    img = AICSImage(czi_path)
    data = img.get_image_data('CZYX')  # img data

    # num of channels and slices from the img, and y and x dimenisons
    num_channels = data.shape[0]
    num_slices = data.shape[1]
    dim_y = data.shape[2]
    dim_x = data.shape[3]

    # empty array to combine data from all channels and slices
    image_stack = np.empty((num_channels * num_slices, dim_y, dim_x), dtype=np.uint16)

    # loop through slices and channels to populate the img stack
    for z in range(num_slices):
        for c in range(num_channels):
            frame = np.array(data[c, z, :, :])  # get the frame at slice z and channel c
            image_stack[z * num_channels + c] = frame  # assign the frame to the appropriate position in the stack

    # voxel size info
    # assuming img.metadata is an Element object
    metadata_xml = img.metadata

    # helper function to safely extract text content from an element and convert to float
    def get_element_float(element, tag):
        child = element.find(tag)
        return float(child.text) if child is not None and child.text is not None else None

    # access common metadata keys from the XML structure
    voxel_size_x = (get_element_float(metadata_xml, './/ScalingX'))* 1_000_000
    voxel_size_y = (get_element_float(metadata_xml, './/ScalingY'))* 1_000_000
    resolution = (voxel_size_x, voxel_size_y)  # define resolution (x, y)

    # save the img stack as .tiff with associated metadata
    output_file_path = os.path.join(output_folder, f'image.tiff')
    tf.imwrite(output_file_path, image_stack, resolution=resolution,
                metadata={'axes': 'ZYX', 'Z': num_channels*num_slices})

    return output_folder  # get the path of the output_folder to be used in the subsequent step 2 as input folder

# '.lif' or '.czi
lif_to_tiff = True #@param{type:"boolean"}

img_dimensions = 2048, 2048 #@param{type:"number"}

czi_to_tiff = False #@param{type:"boolean"}



if lif_to_tiff == True and czi_to_tiff == True:
    print("Please select only one conversion method.")
elif lif_to_tiff == True and czi_to_tiff == False:
    if input_file.lower().endswith('.czi'):
        print('Failed to convert images using lif_to_tiff because the input file has a .czi extension.')
    else:
        output_folder = lif_tiff(base_path, input_file, dimensions=img_dimensions, output_folder_name=output_folder_name)
        if output_folder:
            print(f'Your converted images are in {output_folder}')
        else:
            print('Failed to convert images using lif_to_tiff')
elif lif_to_tiff == False and czi_to_tiff == True:
    if input_file.lower().endswith('.lif'):
        print('Failed to convert images using czi_to_tiff because the input file has a .lif extension.')
    else:
        output_folder = czi_tiff(base_path, input_file, output_folder_name=output_folder_name)
        if output_folder:
            print(f'Your converted images are in {output_folder}')
        else:
            print('Failed to convert images using czi_to_tiff')
elif lif_to_tiff == False and czi_to_tiff == False:
    print('No conversion method selected.')





Your converted images are in /content/gdrive/MyDrive/Colab Notebooks/res/train_up
